### Note:
#### Ignore Gender since it has lot of bias
###### Nan fields:  
state,self_employed,work_interfere,comments
#### Gender
##### For Training data:
Enby,fluid,Genderqueer,Trans-female,Nah,Androgyne,Agender,male leaning androgynous,Trans woman,Neuter,Female (trans),queer,non-binary-  3
Female,female,Cis Female,F,Woman,f,Femake,Female ,woman,cis-female/femme,Female (cis) -2
M,Male,male,m,Male-ish,maile,something kinda male?,Cis Male,Mal,Male ,Make,Guy (-ish) ^_^,Man,msle,Mail,cis male - 1

##### For Test data:
Male,male,m,Malr,Male ,Cis Man - 1
female,F,Female,Woman,femail-2
p,Female (trans),ostensibly male,unsure what that really means - 3


In [16]:
#importing required packages
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn import preprocessing
from sklearn.metrics import accuracy_score,confusion_matrix
from sklearn.model_selection import train_test_split

In [ ]:
#method to count values in a pandas series
def countValues(lab,name):
    idx = pd.Index(lab,name=name)
    return idx.value_counts()

In [11]:
#read data from csv
df = pd.read_csv('trainms.csv')
df.columns

Index(['s.no', 'Timestamp', 'Age', 'Gender', 'Country', 'state',
       'self_employed', 'family_history', 'treatment', 'work_interfere',
       'no_employees', 'remote_work', 'tech_company', 'benefits',
       'care_options', 'wellness_program', 'seek_help', 'anonymity', 'leave',
       'mental_health_consequence', 'phys_health_consequence', 'coworkers',
       'supervisor', 'mental_health_interview', 'phys_health_interview',
       'mental_vs_physical', 'obs_consequence', 'comments'],
      dtype='object')

In [ ]:
#df.info()
countValues(df.work_interfere,"self")
# df[(df['work_interfere']=="Often") & (df['treatment']=="Yes")]

In [12]:
df['work_interfere']=df['work_interfere'].fillna("Maybe")
df['self_employed']=df['self_employed'].fillna("Dont Know")

In [13]:
#Gender Validation
temp = df.Gender
temp = temp.replace(to_replace=["A little about you","something kinda male?","Enby","fluid","Genderqueer","Trans-female","Nah","Androgyne","Agender","male leaning androgynous","Trans woman","Neuter","Female (trans)","queer","non-binary"],value="T")
temp = temp.replace(["Female","female","Cis Female","F","Woman","f","Femake","Female ","woman","cis-female/femme","Female (cis)"],'F')
temp = temp.replace(["M","Male","male","m","Male-ish","maile","Cis Male","Mal","Male ","Make","Guy (-ish) ^_^","Man","msle","Mail","cis male"],'M')
temp.unique()
df.Gender = temp

In [14]:
features = df.drop(columns=['treatment','anonymity','leave','tech_company','Gender','state','Country','comments','s.no','Timestamp'])
y = df.treatment

le = preprocessing.LabelEncoder()
features = features.apply(le.fit_transform)
labels,uniques = pd.factorize(y)

In [17]:
X_train, X_test, y_train, y_test = train_test_split( features, labels, test_size = 0.3, random_state = 100)

In [18]:
#Training the classifier - Dummy Testing

# Create a random forest Classifier. By convention, clf means 'Classifier'
clf = RandomForestClassifier(n_jobs=-1, random_state=0)

# Train the Classifier to take the training features and learn how they relate
# to the training y (the species)
clf.fit(X_train, y_train)

/usr/local/lib/python3.7/dist-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                       max_depth=None, max_features='auto', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=-1,
                       oob_score=False, random_state=0, verbose=0,
                       warm_start=False)

In [19]:
predd = clf.predict(X_train)
accuracy_score(y_train,predd)

0.9942857142857143

In [20]:
predd = clf.predict(X_test)
accuracy_score(y_test,predd)

0.8266666666666667

In [ ]:
# picking specific columns

features = df[['Age','family_history','no_employees','tech_company','benefits','seek_help','obs_consequence']]
y = df.treatment

print(features.info())

le = preprocessing.LabelEncoder()
features = features.apply(le.fit_transform)
labels,uniques = pd.factorize(y)
# Create a random forest Classifier. By convention, clf means 'Classifier'
clf = RandomForestClassifier(n_jobs=-1, random_state=0)

# Train the Classifier to take the training features and learn how they relate
# to the training y (the species)
clf.fit(features, labels)

In [ ]:
#Training the classifier

features = df.drop(columns=['treatment','Gender','state','self_employed','work_interfere','comments','s.no','Timestamp'])
y = df.treatment

le = preprocessing.LabelEncoder()
features = features.apply(le.fit_transform)
labels,uniques = pd.factorize(y)
# Create a random forest Classifier. By convention, clf means 'Classifier'
clf = RandomForestClassifier(n_jobs=-1, random_state=0)

# Train the Classifier to take the training features and learn how they relate
# to the training y (the species)
clf.fit(features, labels)

In [6]:
#Verify predictions
train_predicted = clf.predict(features)

# idx = pd.Index(labels,name="sss")
# print(idx.value_counts())
accuracy = accuracy_score(labels,train_predicted)
print(accuracy)
# print(confusion_matrix(labels,train_predicted))
pd.crosstab(labels, train_predicted, rownames=['Actual'], colnames=['Predicted'])
list(zip(features, clf.feature_importances_))
#                Predicted
#                  No Yes
#  Acutal    No
#            Yes

0.988


[('Age', 0.10972725331553031),
 ('self_employed', 0.014175146562513124),
 ('family_history', 0.08948373854016264),
 ('work_interfere', 0.36326394786441357),
 ('no_employees', 0.052753387335565116),
 ('remote_work', 0.020262978910816142),
 ('benefits', 0.0384789711496224),
 ('care_options', 0.05199447868073637),
 ('wellness_program', 0.024355164608419463),
 ('seek_help', 0.027930747473977134),
 ('mental_health_consequence', 0.03229942330935247),
 ('phys_health_consequence', 0.020813746679903513),
 ('coworkers', 0.029139454402735205),
 ('supervisor', 0.02918852436446976),
 ('mental_health_interview', 0.015006405119990338),
 ('phys_health_interview', 0.03349071027632819),
 ('mental_vs_physical', 0.033555060158251035),
 ('obs_consequence', 0.01408086124721318)]

In [21]:
#Trying to read and view test data
test = pd.read_csv('testms.csv')
temp = test.Age
temp = temp.replace(to_replace=-1,value=22)
test.Age = temp

In [ ]:
# test.info()
# countValues(test.self_employed,"Sefl")
# se = test['self_employed'] == "Yes"
# test[se]

In [22]:
#Validate test gender
temp = test.Gender
temp = temp.replace(to_replace=["Male","male","m","Malr","Male ","Cis Man"],value='M')
temp = temp.replace(to_replace=["female","F","Female","Woman","femail","f"],value='F')
temp = temp.replace(to_replace=["p","Female (trans)","ostensibly male, unsure what that really means"],value='T')
test.Gender = temp

In [23]:
test['work_interfere']=test['work_interfere'].fillna("Maybe")
test['self_employed']=test['self_employed'].fillna("Dont Know")

In [25]:
#predict testing data - Dummy Testing
test_features = test.drop(columns=['anonymity','leave','tech_company','Gender','state','Country','comments','s.no','Timestamp'])
le = preprocessing.LabelEncoder()
test_features = test_features.apply(le.fit_transform)

pred = clf.predict(test_features)
pred

array([0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0,
       1, 1, 0, 0, 1, 0, 1, 1, 1, 0, 0, 0, 1, 0, 0, 1, 1, 1, 0, 1, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 0, 0, 0, 1, 1, 0, 0,
       1, 1, 1, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1,
       0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 1,
       0, 0, 1, 0, 1, 0, 1, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 1,
       0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0,
       0, 1, 1, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 0, 0, 1,
       0, 1, 0, 0, 0, 0, 1, 0, 1, 1, 1, 0, 1, 0, 0, 0, 1, 1, 1, 0, 1, 0,
       0, 1, 0, 0, 1, 1, 1, 1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 1,
       0, 0, 0, 1, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1,
       0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0])

In [ ]:
#predict testing data
test_features = test.drop(columns=['state','Gender','self_employed','work_interfere','comments','s.no','Timestamp'])
le = preprocessing.LabelEncoder()
test_features = test_features.apply(le.fit_transform)

pred = clf.predict(test_features)

In [26]:
# Submission generation
index = [str(i) for i in range(1,len(pred)+1)]
values = ['Yes' if val==0 else 'No' for i,val in enumerate(pred)]
submission_result = pd.DataFrame(values, index=index,columns=['treatment'])
submission_result.index.name = "s.no"

file_name="submission.csv"
submission_result.to_csv(file_name, sep=',')